<a href="https://colab.research.google.com/github/chu-ise/411A-2022/blob/main/notebooks/03/02_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment setup to persist data in Google Drive

In [1]:
import os, sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
      drive.mount("/content/drive")
      !ln -s "/content/drive/My Drive/colab_workspace" workspace
    WORKSPACE_DIR = "/content/workspace"
else:
    WORKSPACE_DIR = "../../workspace" 
print(f'WORKSPACE_DIR = {WORKSPACE_DIR}')
data_dir = os.path.join(WORKSPACE_DIR, "data")
os.makedirs(data_dir, exist_ok=True)

Mounted at /content/drive
WORKSPACE_DIR = /content/workspace


In [2]:
import gdown, os
id = "142Cd7boufftU56nIgbOnqerxWcgQblsO"

data_file = "conf.zip"
conf_dir = data_file.split(".")[0]
if not os.path.exists(conf_dir):
    gdown.download(id=id, output=data_file, quiet=False)
    !unzip $data_file
    os.remove(data_file)
else:
    print(f"{conf_dir} already exists")

Downloading...
From: https://drive.google.com/uc?id=142Cd7boufftU56nIgbOnqerxWcgQblsO
To: /content/conf.zip
100%|██████████| 1.86k/1.86k [00:00<00:00, 1.33MB/s]

Archive:  conf.zip
   creating: conf/
  inflating: conf/config.yaml        
   creating: conf/downloader/
  inflating: conf/downloader/kowiki.yaml  
  inflating: conf/downloader/bok.yaml  
  inflating: conf/downloader/_default.yaml  
  inflating: conf/downloader/enwiki.yaml  


In [3]:
%%capture
%pip install ekorpkit[dataset]
%pip install ekorpkit[wiki]

### Download and Extract Wiki Dump

In [4]:
from ekorpkit.io.fetch.loader.wiki import Wiki

In [5]:
import os
from hydra import initialize, compose

In [6]:
with initialize(config_path="conf"):
    cfg=compose(
        overrides=[
            "+downloader=enwiki",
            f"+data_dir={data_dir}",
        ]
    )
    print(cfg)

{'downloader': {'name': 'enwiki', 'output_dir': '${..data_dir}/${.name}', 'output_file': None, 'autoload': False, 'force_download': False, 'num_workers': 1, 'compress': False, 'verbose': True, '_target_': 'ekorpkit.io.fetch.loader.wiki.Wiki', 'extract': {'_target_': 'extract_wiki'}, 'dump': {'_target_': 'web_download', 'url': 'https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2', 'dump_dir': '${..output_dir}/dump', 'dump_file': '${..name}.xml.bz2'}}, 'data_dir': '/content/workspace/data'}


In [7]:
wk = Wiki(**cfg.downloader)

In [ ]:
wk.download_dump()

[kowiki] download kowiki.xml.bz2: 820MB [02:46, 4.91MB/s]                           


In [ ]:
# wk.extract_wiki()

In [8]:
wk.extract_wiki??

In [9]:
from ekorpkit.io.parse.json import load_jsonlines

wiki_sample_file = '/content/workspace/data/enwiki/wiki_00'
with open(wiki_sample_file) as fo:
  wiki_parsed = load_jsonlines(fo.read())

In [11]:
wiki_parsed[1]

{'id': '12',
 'revid': '13157623',
 'text': 'Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.\nHumans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anar

In [12]:
import pandas as pd

df = pd.DataFrame(wiki_parsed)
df.head()

,id,revid,url,title,text
0,10,20842734,https://en.wikipedia.org/wiki?curid=10,AccessibleComputing,
1,12,13157623,https://en.wikipedia.org/wiki?curid=12,Anarchism,Anarchism is a political philosophy and moveme...
2,13,9784415,https://en.wikipedia.org/wiki?curid=13,AfghanistanHistory,
3,14,9784415,https://en.wikipedia.org/wiki?curid=14,AfghanistanGeography,
4,15,9784415,https://en.wikipedia.org/wiki?curid=15,AfghanistanPeople,
